<a href="https://colab.research.google.com/github/dapopadev/project/blob/main/varias%20transformaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=ca044e8460c64e54628deb7b0a78416656fc6cb430b342e71fa885c3c9bb7b92
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark






**2.INSTALAMOS EL MÓDULO PYSPARK**



3.**CREAMOS LA SESIÓN PYSPARK***

In [2]:
from pyspark.sql import SparkSession

In [3]:
from google.colab import drive

 4. **DISEÑAMOS UNA UNIDAD PERSONAL DE GOOGLE DRIVE**

In [4]:
drive.mount("/drive")

Mounted at /drive


In [5]:
spark=SparkSession.builder.appName("my_app").getOrCreate()

5.***CARGAMOS EL ARCHIVO STOCKS_2021***

In [6]:
df=spark.read.option("Header",True).csv("/drive/MyDrive/CSV_stocks_2021.csv")

 6.**REALIZAMOS UN FILTRO DE LOS DATOS QUE SE CORRESPONDEN CON EL VALOR 'BA' EN LA COLUMNA 'TICKER'**

In [7]:
df_ba=df.filter(df.ticker=='BA')

In [8]:
df_ba.show()

+------+------------------+------------------+------------------+------------------+--------+---------+------------+----------+---+
|ticker|              open|              high|               low|             close|  volume|dividends|stock splits|      date|ccy|
+------+------------------+------------------+------------------+------------------+--------+---------+------------+----------+---+
|    BA|216.24000549316406|216.89999389648438| 212.6999969482422|214.05999755859375|10487600|        0|           0|2020-12-31|USD|
|    BA|             210.0| 210.1999969482422|202.49000549316406|202.72000122070312|21225600|        0|           0|2021-01-04|USD|
|    BA|204.74000549316406|213.35000610351562|204.60000610351562| 211.6300048828125|19338300|        0|           0|2021-01-05|USD|
|    BA|210.22000122070312|215.61000061035156|209.33999633789062|211.02999877929688|16202200|        0|           0|2021-01-06|USD|
|    BA|213.38999938964844|216.60000610351562|211.77999877929688| 212.710006

 **7.IMPORTAMOS LOS OBJETOS DECISIONTREECLASSIFIER Y VECTORASSEMBLER DEL MÓDULO PYSPARK.ML **

In [9]:
from pyspark.ml.classification import DecisionTreeClassifier

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

**8.MODIFICAMOS EL TIPO DE DATOS DE LAS COLUMNAS OPEN,LOW Y CLOSE A FLOAT

In [12]:
df=df.withColumn('open',df.open.cast('float'))

In [13]:
df=df.withColumn('low',df.low.cast('float'))

In [14]:
df=df.withColumn('close',df.close.cast('float'))

In [15]:
df.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- open: float (nullable = true)
 |-- high: string (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: string (nullable = true)
 |-- dividends: string (nullable = true)
 |-- stock splits: string (nullable = true)
 |-- date: string (nullable = true)
 |-- ccy: string (nullable = true)



**9.CREAREMOS UNA COLUMNA'FEATURES' UTILIZANDO LAS COLUMNAS OPEN,LOW,Y CLOSE MEDIANTE VECTORASSEMBLER**

In [16]:
va=VectorAssembler(inputCols=['open','low','close'],outputCol='features')

In [17]:
va_df=va.transform(df)

In [18]:
va_df.show()

+------+---------+------------------+---------+---------+-------+---------+------------+----------+---+--------------------+
|ticker|     open|              high|      low|    close| volume|dividends|stock splits|      date|ccy|            features|
+------+---------+------------------+---------+---------+-------+---------+------------+----------+---+--------------------+
|   HON|207.45502|209.42141232111007|206.14735|209.12645|1406400|      0.0|           0|2020-12-31|USD|[207.455017089843...|
|   HON| 209.2641|209.43123817541044|202.89297|204.45625|2328900|      0.0|           0|2021-01-04|USD|[209.264099121093...|
|   HON|203.50256| 206.6586329757187|203.50256| 204.9577|2172100|      0.0|           0|2021-01-05|USD|[203.502563476562...|
|   HON|205.93106|210.38495140642195|205.71475|208.69385|2747900|      0.0|           0|2021-01-06|USD|[205.931060791015...|
|   HON|209.31328|210.41445198270796|207.25839|209.03798|2057300|      0.0|           0|2021-01-07|USD|[209.313278198242...|


**10.CONVERTIMOS LA COLUMNA 'TICKER' A TIPO NUMÉRICO, UTILIZANDO STRINGINDEXER Y LLAMARLA'LABEL'**

In [19]:
from pyspark.ml.feature import StringIndexer

In [20]:
df=df.withColumn('ticker',df.ticker.cast('int'))

In [21]:
df=df.withColumnRenamed('ticker','label')

In [22]:
df.printSchema()

root
 |-- label: integer (nullable = true)
 |-- open: float (nullable = true)
 |-- high: string (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: string (nullable = true)
 |-- dividends: string (nullable = true)
 |-- stock splits: string (nullable = true)
 |-- date: string (nullable = true)
 |-- ccy: string (nullable = true)



**11.CREAMOS EL OBJETO DECISSIONTREECLASSIFIER**

In [23]:
dtc=DecisionTreeClassifier(featuresCol='features',labelCol='label')

**12. ENTRENAMOS EL MODELO UTILIZANDO EL 20% DEL DATAFRAME COMO DATOS DEL TRAINING Y EL 80% RESTANTE COMO DATOS DEL TEST**

In [24]:
(train,test)=va_df.randomSplit([0.2,0.8])

In [25]:
train.count()

1033

In [26]:
train.show()

+------+----------+------------------+----------+----------+---------+---------+------------+----------+---+--------------------+
|ticker|      open|              high|       low|     close|   volume|dividends|stock splits|      date|ccy|            features|
+------+----------+------------------+----------+----------+---------+---------+------------+----------+---+--------------------+
|  AAPL| 121.08532|122.01100143225499| 120.16959| 120.82652| 80819200|      0.0|           0|2021-03-29|USD|[121.085319519042...|
|  AAPL| 122.75752|123.66329726241875|121.573044| 121.97119| 95467100|      0.0|           0|2021-03-23|USD|[122.757522583007...|
|  AAPL| 122.79654|124.55133845622976| 122.49742| 124.32202| 92612000|      0.0|           0|2021-05-19|USD|[122.796539306640...|
|  AAPL|124.230644|125.12646681524532| 121.27443| 121.49341|112966300|      0.0|           0|2021-03-03|USD|[124.230644226074...|
|  AAPL| 124.31205|124.48154658984885|122.766624|123.175415| 76229200|      0.0|          

In [27]:
test.show()

+------+----------+------------------+----------+----------+---------+---------+------------+----------+---+--------------------+
|ticker|      open|              high|       low|     close|   volume|dividends|stock splits|      date|ccy|            features|
+------+----------+------------------+----------+----------+---------+---------+------------+----------+---+--------------------+
|  AAPL| 118.47748|121.49341458486911|118.238594|120.527916|129525800|      0.0|           0|2021-03-09|USD|[118.477478027343...|
|  AAPL|118.985115|121.09527579793972| 118.44762|120.030235| 98844700|      0.0|           0|2021-03-25|USD|[118.985115051269...|
|  AAPL|119.343445| 120.8663397025217|119.124466| 119.43302|185549500|      0.0|           0|2021-03-19|USD|[119.343444824218...|
|  AAPL| 119.55246| 119.8411162426809|118.308266| 119.34344| 85671900|      0.0|           0|2021-03-30|USD|[119.552459716796...|
|  AAPL|119.771454|123.29502075216398|119.701775|122.817245|111912300|      0.0|          

In [28]:
test.count()

4027

**13.CONVERTIMOS DE NUEVO LOS VALORES DE'TICKER A SU ESTADO ORIGINAL CON INDEXTOSTRING**

In [29]:
df.printSchema()

root
 |-- label: integer (nullable = true)
 |-- open: float (nullable = true)
 |-- high: string (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: string (nullable = true)
 |-- dividends: string (nullable = true)
 |-- stock splits: string (nullable = true)
 |-- date: string (nullable = true)
 |-- ccy: string (nullable = true)



In [30]:
df.show()

+-----+---------+------------------+---------+---------+-------+---------+------------+----------+---+
|label|     open|              high|      low|    close| volume|dividends|stock splits|      date|ccy|
+-----+---------+------------------+---------+---------+-------+---------+------------+----------+---+
| null|207.45502|209.42141232111007|206.14735|209.12645|1406400|      0.0|           0|2020-12-31|USD|
| null| 209.2641|209.43123817541044|202.89297|204.45625|2328900|      0.0|           0|2021-01-04|USD|
| null|203.50256| 206.6586329757187|203.50256| 204.9577|2172100|      0.0|           0|2021-01-05|USD|
| null|205.93106|210.38495140642195|205.71475|208.69385|2747900|      0.0|           0|2021-01-06|USD|
| null|209.31328|210.41445198270796|207.25839|209.03798|2057300|      0.0|           0|2021-01-07|USD|
| null|209.22478|209.82453011896777|204.18097|206.50131|3278900|      0.0|           0|2021-01-08|USD|
| null|204.61356|206.09818517407587|204.33827|204.85936|2938900|      0.0

**14.UNIMOS MEDIANTE UN INNER JOIN EL DATAFRAME CON LAS PREDICCIONES Y EL QUE CONTIENE LOS VALORES REESTABLECIDOS DE LA COLUMNA 'TICKER' ** **bold text**

In [31]:
spark=SparkSession.builder.getOrCreate()

In [32]:
df=spark.read.csv("/drive/MyDrive/CSV_stocks_2021.csv",header=True,inferSchema=True)

In [33]:
va_df=spark.read.csv("/drive/MyDrive/CSV_stocks_2021.csv",header=True,inferSchema=True)

In [34]:
join_column=['ticker','features']